# Visualizations for Presentations

In [1]:
# import xarrayutils.visualization as viz
import xarrayutils as xut
from xarrayutils.visualization import Movie
from xmitgcm import open_mdsdataset
from mitgcm_surface_tracer.tracer_processing import tracer_engine,readbin
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

/rigel/home/jb3210/miniconda/envs/standard/lib/python2.7/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [2]:
# put these into xarrayutils
def harm_mean(da, dim=None, axis=None):
    if dim is not None:
        if axis is not None:
            raise ValueError('cannot set both `dim` and `axis`')
        if isinstance(da, xr.Dataset):
            axis = da[da.data_vars.keys()[0]].get_axis_num(dim)
        elif isinstance(da, xr.DataArray):
            axis = da.get_axis_num(dim)
    if isinstance(da,np.ndarray):
        da[~xr.ufuncs.isfinite(da)] = np.nan
        da[abs(da) == 0.0] = np.nan
        a = 1.0/da
        out = 1.0/(np.nanmean(a,axis=axis))
    elif isinstance(da,(xr.DataArray,xr.Dataset)):
        da = da.where(xr.ufuncs.isfinite(da))
        da = da.where(abs(da) > 0.0)
        a = 1.0/da
        out = 1.0/(a.mean(dim))
    else:
        raise RuntimeError('data type of da not supported')
    return out

In [3]:
root = '/rigel/ocp/users/jb3210/aviso_surface_tracer/runs'
run = 'run_KOC_LAT'
odir = '/rigel/ocp/users/jb3210/aviso_surface_tracer/videos'

In [ ]:
from xgcm import Grid
from xarrayutils.build_grids import rebuild_grid
viddir = odir
fname = root+'/'+run+'/output/KOC_RAW.nc'
ds = xr.open_dataset(fname,chunks={'time':1})

# trac = ds.TRAC01.mean('tracernum')
trac = ds.TRAC01.isel(tracernum=1)
trac.coords['XC'] = ds['XC']
trac.coords['YC'] = ds['YC']

ds_re = rebuild_grid(ds)
grid = Grid(ds_re)
grad = xr.ufuncs.sqrt(xut.xmitgcm_utils.gradient_sq_amplitude(grid,trac))
grad = grad.isel(time=slice(0,10))
grad.coords['XC'] = ds['XC']
grad.coords['YC'] = ds['YC']
grad = grad.where(ds.mask)

In [ ]:
trac.isel(time = 5).plot(robust=True)

In [ ]:
%time
Movie(trac,viddir,clim = np.array([-90,90]),moviename='LAT_raw',
          plot_style='map',
          lon='XC',lat='YC',
          cmap=plt.cm.inferno,
          bgcolor='0.95',
          dpi=72)

In [ ]:
%time
Movie(grad,viddir,clim = np.array([1e-7,1e-4]),moviename='LAT_Grad',
          norm=mpl.colors.SymLogNorm(10),
          plot_style='map',
          lon='XC',lat='YC',
          cmap=plt.cm.Blues,
          bgcolor='0.95',
          dpi=72)